In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import random

In [5]:
#create url list
chromedriver = "/Users/manveersadhal/Downloads/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [6]:
#homepage for e46 m3 auctions
e46_m3_url = 'https://bringatrailer.com/bmw/e46-m3/' 

In [7]:
driver = webdriver.Chrome(chromedriver)
driver.get(e46_m3_url)
time.sleep(2)

In [9]:
soup_initial = BeautifulSoup(driver.page_source, 'html5lib')
total_completed_auctions = int(soup_initial.find('span', attrs = {'data-bind' : "text: formattedTotal"}).string)
initial_auctions_loaded = 32
items_per_click = 32
times_to_click = (total_completed_auctions - initial_auctions_loaded) // items_per_click + 1

show_more_button = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/div/div[6]/div[3]/div[4]/button/span[1]')

for i in range(times_to_click):
    try:
        show_more_button.click()
    except StaleElementReferenceException:
        print(f'{e46_m3_url} - no more auctions to load')
    time.sleep(5)
        
soup_fully_loaded = BeautifulSoup(driver.page_source, 'html5lib')

In [12]:
soup_e46_main = BeautifulSoup(driver.page_source, 'html5lib')

In [14]:
url_anchor_list = soup_e46_main.find_all('a', attrs={'data-bind' : "attr: { href: url }"})

In [15]:
url_list = [link['href'] for link in url_anchor_list]

In [17]:
e46_m3_url_list_df = pd.DataFrame(url_list)

In [18]:
e46_m3_url_list_df.to_csv('e46_m3_url_list', header = False)

In [19]:
e36_m3_url = 'https://bringatrailer.com/bmw/e36-m3/' 

In [220]:
e90_e92_url = 'https://bringatrailer.com/bmw/e90-e92-m3/'

In [222]:
driver = webdriver.Chrome(chromedriver)
driver.get(e90_e92_url)
time.sleep(2)

In [223]:
soup_initial = BeautifulSoup(driver.page_source, 'html5lib')
total_completed_auctions = int(soup_initial.find('span', attrs = {'data-bind' : "text: formattedTotal"}).string)
initial_auctions_loaded = 32
items_per_click = 32
times_to_click = (total_completed_auctions - initial_auctions_loaded) // items_per_click + 1

show_more_button = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/div/div[2]/div[3]/div[4]/button/span[1]')

for i in range(times_to_click):
    try:
        show_more_button.click()
    except StaleElementReferenceException:
        print(f'{e90_e92_m3_url} - no more auctions to load')
    time.sleep(5)
        
soup_fully_loaded = BeautifulSoup(driver.page_source, 'html5lib')

In [224]:
soup_e90_e92_main = BeautifulSoup(driver.page_source, 'html5lib')

In [225]:
url_anchor_list = soup_e90_e92_main.find_all('a', attrs={'data-bind' : "attr: { href: url }"})

In [226]:
url_list = [link['href'] for link in url_anchor_list]

In [229]:
e90_e92_m3_url_list_df = pd.DataFrame(url_list)

In [230]:
e90_e92_m3_url_list_df.to_csv('e90_e92_m3_url_list', header = False)

In [25]:
e36_m3_url_list_df = pd.DataFrame(url_list)

In [26]:
e36_m3_url_list_df.to_csv('e36_m3_url_list', header = False)

In [231]:
driver.quit()

In [257]:
def get_body_style(title):
    if 'convertible' in title.text.lower():
        body_style = 'Convertible'
    else:
        body_style = 'Hard top'
    return body_style


def get_model_year(title):
    if re.search(r'\d{4}', title.text):
        match_str = re.search(r'\d{4}', title.text)
        return datetime.strptime(match_str.group(), '%Y').date().year
    else:
        return 'Unknown'


def get_mileage(essentials_list):
    keywords = ['mile', 'km', 'kilometer', 'kilometre']
    antikeywords = ['seller', 'location']
    for item in essentials_list:
        if any(word in str(item).lower() for word in keywords) and not (any(word in str(item).lower() for word in antikeywords)):
            temp = BeautifulSoup(str(item))
            mileage = temp.get_text()
            return mileage
    return 'Unknown'

    
def get_transmission(essentials_list):
    keywords = ['transmission', 'manual', 'smg', 'sequential', 'gearbox', 'trans', 'dct', 'clutch']
    for item in essentials_list:
        if any(word in str(item).lower() for word in keywords):
            temp = BeautifulSoup(str(item))
            transmission = temp.get_text()
            return transmission
    return 'Unknown'


def get_VIN(essentials_list):
    keywords = ['chassis', 'vin']
    antikeywords = ['seller', 'location']
    for item in essentials_list:
        if any(word in str(item).lower() for word in keywords) and not (any(word in str(item).lower() for word in antikeywords)):
            temp = BeautifulSoup(str(item))
            VIN = temp.get_text()
            return VIN
    return 'Unknown'


def get_seller_type(essentials_list):
    keywords = ['private party or dealer']
    for item in essentials_list:
        if any(word in str(item).lower() for word in keywords):
            temp = BeautifulSoup(str(item))
            seller_type = temp.get_text()
            return seller_type
    return 'Unknown'


def get_location(essentials_list):
    keywords = ['location']
    for item in essentials_list:
        if any(word in str(item).lower() for word in keywords):
            temp = BeautifulSoup(str(item))
            location = temp.get_text()
            return location
    return 'Unknown'


def get_interior_color(essentials_list):
    keywords = ['leather', 'cloth', 'alcantara', 'upholster', 'cinnamon']
    antikeywords = ['metallic', 'paint']
    for item in essentials_list:
        if any(word in str(item).lower() for word in keywords):
            temp = BeautifulSoup(str(item)).get_text().lower()
            if 'over' in temp:
                interior_color = temp.split('over')[1].strip()
                return interior_color
            elif '/' in temp:
                interior_color = temp.split('/')[1].strip()
                return interior_color
            elif not any(word in temp for word in antikeywords):
                interior_color = temp
                return interior_color
    return 'Unknown'


def get_exterior_color(essentials_list):
    keywords = ['black', 'white', 'jet', 'carbon', 'sapphire', 'silver', 'gray', 'grey', 'blue', 'topaz', 'mystic', 'interlagos', 'fern', 'oxford', 'green', 'imola', 'laguna seca', 'metallic', 'phoenix', 'yellow', 'red', 'purple', 'violet', 'hellrot', 'paint', 'orange']
    antikeywords = ['nappa', 'leather', 'upholstery', 'location', 'seller', 'bluetooth', 'transmission', 'differential']
    for item in essentials_list:
        if any(word in str(item).lower() for word in keywords):
            temp = BeautifulSoup(str(item)).get_text().lower()
            if 'over' in temp:
                exterior_color = temp.split('over')[0].strip()
                return exterior_color
            elif '/' in temp:
                exterior_color = temp.split('/')[0].strip()
                return exterior_color
            elif not any(word in temp for word in antikeywords):
                exterior_color = temp
                return exterior_color
    return 'Unknown'


def get_date_sold(soup):
    month_sold, day_sold, year_sold = soup.find('div', class_ = 'listing-available-info').find_all('span', class_ = 'data-value')[1].string.split('/')
    month_sold = month_sold.rjust(2, '0')
    day_sold = day_sold.rjust(2, '0')
    year_sold = '20' + year_sold
    date_sold = datetime.strptime(f'{month_sold}/{day_sold}/{year_sold}', '%m/%d/%Y')
    return date_sold

def get_price_and_auction_result(soup):
    if 'sold' in soup.find('span', class_ = 'data-label').text.lower():         
        price = float(soup.find('span', class_ = 'data-value price').text[1:].replace(',',''))
        auction_result = 'Sold'
    elif soup.find('span', class_ = 'data-label').text.lower() == 'bid to':
        price = soup.find('span', class_ = 'data-value').text[1:].replace(',','')
        auction_result = 'Reserve not met'
    else:
        price = soup.find('span', class_ = 'data-value').text[1:].replace(',','')
        auction_result = 'Ended'
    return price, auction_result

def get_num_photos(soup):
    num_photos = len(soup.find_all('img', attrs={'data-bind' : "attr: { src: images.small.url }"}))
    if num_photos >= 60: #max photos in gallery is 60, but sometimes photos in comments are caught in this search
        if soup.find('span', attrs = {'data-bind' : "click: $parent.show, clickBubble: false, css: { show: isLastItem }, text: $parent.moreText"}):
            additional_photos = int(soup.find('span', attrs = {'data-bind' : "click: $parent.show, clickBubble: false, css: { show: isLastItem }, text: $parent.moreText"}).string[1:])
            num_photos = 60 + additional_photos
    return num_photos

def scrape_data(url_list):
    chromedriver = "/Users/manveersadhal/Downloads/chromedriver" # path to the chromedriver executable
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)
    cars_df = pd.DataFrame(columns = ['URL', 'VIN', 'YEAR', 'MILEAGE', 'EXT_COLOR', 'INT_COLOR', 'TRANSMISSION', 'BODY_STYLE', 'LOCATION', 'DATE_SOLD', 'SELLER_TYPE', 'NUM_PHOTOS', 'SALE_PRICE', 'RESULT'])
    for i, url in enumerate(url_list):
        driver.get(url)
        wait = 3 + 2*random.random()
        time.sleep(wait)
        soup = BeautifulSoup(driver.page_source, 'html5lib')
        
        if 'withdrawn' in soup.find('span', class_ = 'data-label').text.lower():
            print(f'skip {url} - withdrawn auction')
            continue
        
        essentials_list = soup.find('ul', class_ = 'listing-essentials-items').contents
        listing_title = soup.find('h1', class_ = 'post-title listing-post-title')
        price, auction_result = get_price_and_auction_result(soup)
        
        #save html for each page to avoid having to scrape again
        stripped_url = url.replace('https://bringatrailer.com/listing/','')[:-1]
        filename = './html/' + stripped_url + '.html'
        with open(filename, 'w') as f:
            f.write(soup.text)
        
        #append data from current page to DF
        cars_df = cars_df.append({'URL' : url,
               'VIN' : get_VIN(essentials_list),
               'YEAR' : get_model_year(listing_title),
               'MILEAGE' : get_mileage(essentials_list),
               'EXT_COLOR' : get_exterior_color(essentials_list),
               'INT_COLOR' : get_interior_color(essentials_list),
               'TRANSMISSION' : get_transmission(essentials_list),
               'BODY_STYLE' : get_body_style(listing_title),
               'LOCATION' : get_location(essentials_list),
               'DATE_SOLD' : get_date_sold(soup),
               'SELLER_TYPE' : get_seller_type(essentials_list),
               'NUM_PHOTOS' : get_num_photos(soup),
               'SALE_PRICE' : price,
                'RESULT' : auction_result}, ignore_index = True)
        
        #pause after each 25 pages and write to csv
        if (i + 1) % 25 == 0:
            time.sleep(5)
            cars_df.to_csv('cars_temp.csv')
        
        print(f'completed loop {i}')
    cars_df.to_csv(f"cars-{datetime.now().strftime('%Y-%m-%d %H-%M-%S')}.csv")
    driver.quit()
        

In [144]:
e46_m3_urls_df = pd.read_csv('e46_m3_url_list.csv', header = None)

In [145]:
e46_m3_urls = e46_m3_urls_df.iloc[:][1].tolist()

In [147]:
len(e46_m3_urls)

539

In [148]:
e36_m3_urls_df = pd.read_csv('e36_m3_url_list.csv', header = None)

In [149]:
e36_m3_urls = e36_m3_urls_df.iloc[:][1].tolist()

In [150]:
len(e36_m3_urls)

401

In [239]:
e90_e92_m3_urls_df = pd.read_csv('e90_e92_m3_url_list.csv', header = None)

In [240]:
e90_e92_m3_urls = e90_e92_m3_urls_df.iloc[:][1].tolist()

In [241]:
len(e90_e92_m3_urls)

181

In [242]:
url_list = e36_m3_urls + e46_m3_urls + e90_e92_m3_urls

In [243]:
len(url_list)

1121

In [252]:
scrape_data(url_list)

completed loop 0
completed loop 1
completed loop 2
completed loop 3
completed loop 4
completed loop 5
completed loop 6
completed loop 7
completed loop 8
completed loop 9
completed loop 10
completed loop 11
completed loop 12
completed loop 13
completed loop 14
completed loop 15
completed loop 16
completed loop 17
completed loop 18
completed loop 19
completed loop 20
completed loop 21
completed loop 22
completed loop 23
completed loop 24
completed loop 25
completed loop 26
completed loop 27
completed loop 28
completed loop 29
completed loop 30
completed loop 31
completed loop 32
completed loop 33
completed loop 34
completed loop 35
completed loop 36
completed loop 37
completed loop 38
completed loop 39
completed loop 40
completed loop 41
completed loop 42
completed loop 43
completed loop 44
completed loop 45
completed loop 46
completed loop 47
completed loop 48
completed loop 49
completed loop 50
completed loop 51
completed loop 52
completed loop 53
completed loop 54
completed loop 55
co

completed loop 432
completed loop 433
completed loop 434
completed loop 435
completed loop 436
completed loop 437
completed loop 438
completed loop 439
completed loop 440
completed loop 441
skip https://bringatrailer.com/listing/2003-bmw-m3-coupe-126/ - withdrawn auction
completed loop 443
completed loop 444
completed loop 445
completed loop 446
completed loop 447
completed loop 448
completed loop 449
completed loop 450
completed loop 451
completed loop 452
completed loop 453
completed loop 454
completed loop 455
completed loop 456
completed loop 457
completed loop 458
completed loop 459
completed loop 460
completed loop 461
completed loop 462
completed loop 463
completed loop 464
completed loop 465
completed loop 466
completed loop 467
completed loop 468
completed loop 469
completed loop 470
completed loop 471
completed loop 472
completed loop 473
completed loop 474
completed loop 475
completed loop 476
completed loop 477
completed loop 478
completed loop 479
completed loop 480
comple

completed loop 854
completed loop 855
completed loop 856
completed loop 857
completed loop 858
completed loop 859
completed loop 860
completed loop 861
completed loop 862
completed loop 863
completed loop 864
completed loop 865
completed loop 866
completed loop 867
skip https://bringatrailer.com/listing/2003-bmw-m3-coupe-29/ - withdrawn auction
completed loop 869
completed loop 870
completed loop 871
completed loop 872
completed loop 873
completed loop 874
completed loop 875
completed loop 876
completed loop 877
completed loop 878
completed loop 879
completed loop 880
completed loop 881
completed loop 882
completed loop 883
completed loop 884
completed loop 885
completed loop 886
completed loop 887
completed loop 888
completed loop 889
completed loop 890
completed loop 891
completed loop 892
completed loop 893
completed loop 894
completed loop 895
completed loop 896
completed loop 897
completed loop 898
completed loop 899
completed loop 900
completed loop 901
completed loop 902
complet